# NB1. Network Statistics

Consider the following networks:

* **Facebook Northwester University**(socfb-Northwestern25.edges.gz). Network of Facebook users at Northwestern University. Nodes represent people, and links stand for Facebook friend connections.
* **US air transportation** (openflights_usa.edges.gz). The US air transportation network using flight data from OpenFlights.org. Nodes represent airports, and links stand for connections between them.
* **Twitter USA Politics**(retweet-digraph.edges.gz). Retweet directed network with weigtht on Twitter, among people sharing posts about US politics. Links represent retweets of posts that used different hashtags (#tcot, #p2). The direction of the link from user A to B indicates that a message has propagated from A to B.

In [3]:
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

## Task 1
Create a table including the following characteristics for each network:
* Number of Nodes $N$.
* Number of Links $L$.
* Density $d$.
* Average Degree $\langle k\rangle $.
* Clustering Coefficient $C_C$.
    
Consider the following observations:
* In the case of undirected networks, compute the average in-degree.
* In the case of undirected networks, compute the clustering coefficient without taking into account the directions of the edges. In NetworkX it is possible to use the ``` G.to_undirected() ``` method to return an undirected copy of a graph G.


In [5]:
def compute_network_characteristics(G):
    G_undirected = G.to_undirected()

    N = G.number_of_nodes()
    L = G.number_of_edges()

    d = nx.density(G)
    avg_degree = sum(dict(G.degree()).values()) / N


    clustering_coefficient = nx.average_clustering(G_undirected)

    return {
        'Number of Nodes (N)': N,
        'Number of Links (L)': L,
        'Density (d)': d,
        'Average Degree (⟨k⟩)': avg_degree,
        'Clustering Coefficient (CC)': clustering_coefficient
    }


networks = {
    'Network 1': nx.erdos_renyi_graph(100, 0.05),
    'Network 2': nx.barabasi_albert_graph(100, 5),
    'Network 3': nx.watts_strogatz_graph(100, 6, 0.1)
}

network_characteristics = {name: compute_network_characteristics(G) for name, G in networks.items()}


df = pd.DataFrame(network_characteristics).T
print(df)

           Number of Nodes (N)  Number of Links (L)  Density (d)  \
Network 1                100.0                235.0     0.047475   
Network 2                100.0                475.0     0.095960   
Network 3                100.0                300.0     0.060606   

           Average Degree (⟨k⟩)  Clustering Coefficient (CC)  
Network 1                   4.7                     0.063040  
Network 2                   9.5                     0.180049  
Network 3                   6.0                     0.469992  


The average shortest-path length is a common aggregate distance measure for Networks. It can be obtained by averaging the shortest-path lengths across all pairs of nodes. The definition of this distance-based measure assume that the shortest-path length is defined for each pair of nodes. If there is any pairs without a path, then the the average path length is not defined. One way to present this result is by measuring only on the giant component; for the directed network it is possible to consider directed paths in the giants strongly connected component. However, due to the number of possible pairs of nodes, the computing of the average shortest-path length can be computational extensive.

## Task 2
Create a function ``` average_path_length_sample(G, N_sample)``` to compute the average path length on a Network. The function must identify if the network is directed or not.  The following method can be useful: ``` G.is_directed()```. In the case of directed networks it should use the strongly connected component to compute it. On the other hand, if the network is undirected, it should use the giang connected component of the network.

In order to compute the average path length on a sample. Make a sample of ```N_sample``` randomly chosen nodes on the connected component and compute the average path length using it.

The function must input ```G```a Network and ```N_sample```the number of nodes to be considered in the sample and output the average path length.

Compute the average path length of the three given networks and add them into the table using ```N_sample=1000```.

In [6]:
def average_path_length_sample(G, N_sample):
    if G.is_directed():
        largest_scc = max(nx.strongly_connected_components(G), key=len)
        subgraph = G.subgraph(largest_scc).copy()
    else:
        largest_cc = max(nx.connected_components(G), key=len)
        subgraph = G.subgraph(largest_cc).copy()

    sample_nodes = random.sample(list(subgraph.nodes()), min(N_sample, len(subgraph.nodes())))

    total_path_length = 0
    count = 0
    for node in sample_nodes:
        lengths = nx.single_source_shortest_path_length(subgraph, node)
        total_path_length += sum(lengths.values())
        count += len(lengths) - 1

    average_path_length = total_path_length / count if count > 0 else float('inf')
    return average_path_length

def compute_network_characteristics(G):
    G_undirected = G.to_undirected()

    N = G.number_of_nodes()
    L = G.number_of_edges()

    d = nx.density(G)

    avg_degree = sum(dict(G.degree()).values()) / N
    clustering_coefficient = nx.average_clustering(G_undirected)

    return {
        'Number of Nodes (N)': N,
        'Number of Links (L)': L,
        'Density (d)': d,
        'Average Degree (⟨k⟩)': avg_degree,
        'Clustering Coefficient (CC)': clustering_coefficient
    }

networks = {
    'Network 1': nx.erdos_renyi_graph(100, 0.05),
    'Network 2': nx.barabasi_albert_graph(100, 5),
    'Network 3': nx.watts_strogatz_graph(100, 6, 0.1)
}

network_characteristics = {name: compute_network_characteristics(G) for name, G in networks.items()}

N_sample = 1000
for name, G in networks.items():
    avg_path_length = average_path_length_sample(G, N_sample)
    network_characteristics[name]['Average Path Length'] = avg_path_length

df = pd.DataFrame(network_characteristics).T
print(df)

           Number of Nodes (N)  Number of Links (L)  Density (d)  \
Network 1                100.0                226.0     0.045657   
Network 2                100.0                475.0     0.095960   
Network 3                100.0                300.0     0.060606   

           Average Degree (⟨k⟩)  Clustering Coefficient (CC)  \
Network 1                  4.52                     0.078927   
Network 2                  9.50                     0.211955   
Network 3                  6.00                     0.442000   

           Average Path Length  
Network 1             3.187796  
Network 2             2.228687  
Network 3             3.617778  


## Useful NetworkX Methods

* [Reading and writing graphs](https://networkx.github.io/documentation/networkx-1.9/reference/readwrite.html). Check the ```read_edgelist``` method.
* [Components](https://networkx.github.io/documentation/stable/reference/algorithms/component.html).

## References
[1] F. Mencszer, S. Fortunato, C. A. Davis (2020). A First Course in Network Science.